# Introduction to `tidyverse` Round 2
An ["opinionated" collection of R packages](https://tidyverse.org) for data science, driven by a coherent underlying design philosophy.
These packages are meant to help you with two essential processes:
1. **Data clean-up and organization**: Structure should be intuitive, so that it's easy to model, manipulate, and think about the data
2. **Data plotting**: The grammar of graphics (Week 13)

In [1]:
library(tidyverse)

-- Attaching core tidyverse packages ------------------------ tidyverse 2.0.0 --
v dplyr     1.1.4     v readr     2.1.5
v forcats   1.0.0     v stringr   1.5.1
v ggplot2   3.5.2     v tibble    3.3.0
v lubridate 1.9.4     v tidyr     1.3.1
v purrr     1.1.0     
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
i Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


## Data preparation

### Loading the data

In [2]:
data <- read_csv("data/148338_220209_095045_M057814.csv", skip=2)

Rows: 241 Columns: 32
-- Column specification --------------------------------------------------------
Delimiter: ","
chr (21): type, stim1, stim2, stimPos, stimFormat, feedbackIncorrect, head, ...
dbl (11): rowNo, ITI, feedbackTime, random, ITI_ms, ITI_f, ITI_fDuration, ti...

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [3]:
education_level <- data %>% pull(response) %>% first()  

data <- data %>%
    # Keep only useful columns
    select(c(rowNo, type, stim1, stim2, stimPos, trialType, response, RT)) %>%
    
    # Keep only useful rows
    filter(type != "form") %>%
    
    # Add demographic and trial-number info, turn trial type to factor
    mutate(
        education_level = education_level, # Add info
        trial_number = row_number(),
        trialType = factor(trialType, levels = c("incongruent", "congruent"))
    ) %>%
    
    # Rename trialType to trial_type
    rename(trial_type = trialType)

head(data)

rowNo,type,stim1,stim2,stimPos,trial_type,response,RT,education_level,trial_number
<dbl>,<chr>,<chr>,<chr>,<chr>,<fct>,<chr>,<dbl>,<chr>,<int>
190,test,croissantLarge,breadSmall,-341 0; 341 0,incongruent,j,729,College or Technical School,1
125,test,lampSmall,shelfLarge,-235 0; 235 0,congruent,f,625,College or Technical School,2
200,test,schoolbusLarge,bikeSmall,-370 0; 370 0,congruent,j,1477,College or Technical School,3
152,test,orangeLarge,raspberrySmall,-307 0; 307 0,congruent,j,793,College or Technical School,4
110,test,hammerLarge,ladderSmall,-156 0; 156 0,incongruent,j,900,College or Technical School,5
215,test,wheelSmall,coinLarge,-300 0; 300 0,incongruent,f,699,College or Technical School,6


### Tidying the data

Our data is almost tidy, but we still need to do several things.

#### Exercise 1
There are two columns that are uninformative. Remove them.

In [4]:
data <- data %>% select(-c(rowNo, type))
# head(data)

#### Exercise 2
The columns `stim1` and `stim2` refer to the stimulus presented on the left and, respectively, right. Rename the columns to make them more informative.

In [5]:
data <- data %>% rename(stim_left = stim1, stim_right = stim2)

#### Exercise 3
Add a new column called `subject_id` and set this participant to 1.

In [6]:
data <- data %>% mutate(subject_id = 1)
# data

#### Exercise 4
Add a new column called `correct_side`. This column should encode the side of the correct answer (equivalently, the side of the smaller image). Use `str_detect(string, pattern)` to compute where the smaller image was on the screen and use a conditional `mutate` to fill in the values of `correct_side`.

In [7]:
data <- data %>% 
    mutate(correct_side = if_else(str_detect(stim_left, "small"), "left", "right"))

#### Exercise 5
Add a new column called `correct_key`. This column should be equal to `f` if the smaller image was on the left, and to `j` if the smaller image was on the right.

In [8]:
data <- data %>% 
    mutate(correct_key = if_else(correct_side == "left", "f", "j"))

#### Exercise 6
Add a new column called `correct`. This column should be equal to 1 if the participant provided a correct response, 0 otherwise.

In [9]:
data <- data %>% 
    mutate(correct = if_else(response == correct_key, 1, 0))

#### Exercise 7
The experiment had 240 trials, equally divided into two blocks. Add a new column called `trial_block` that encodes this.

In [10]:
data <- data %>% 
    mutate(trial_block = if_else(trial_number <= 120, 1, 2))

#### Exercise 8
Recode `trial_number` so that it codes the trial number within a block. Instead of going from 1 to 240, it should go from 1 to 120 twice.

In [11]:
data <- data %>% 
    mutate(trial_number = if_else(trial_number > 120, trial_number - 120, trial_number))

#### Exercise 9
It is good practice to use a consistent style throughout your script. One such style is called **snake**, the standard in python, which uses only lowercase letters and underscores: `variable_name`. Another common style is **camel**, the standard in JavaScript, which uses capital letters to mark the beginning of a new word: `variableName`. Our tibble at this point uses both styles, so turn all variable names to snake case. You can use `colnames` to see the vector of column names.

In [12]:
data <- data %>% 
    rename(stim_pos = stimPos, rt = RT)

#### Exercise 10
Using `select`, reorder the columns such that participant information comes first, followed by trial block and number, followed by trial info.

In [13]:
data <- data %>% 
    select(subject_id, trial_block, trial_number, everything())
data %>% colnames()

[1] "subject_id"      "trial_block"     "trial_number"    "stim_left"      
 [5] "stim_right"      "stim_pos"        "trial_type"      "response"       
 [9] "rt"              "education_level" "correct_side"    "correct_key"    
[13] "correct"

#### Exercise 11
Concatenate all the commands in Exercises 1–10 into a single cell and store the output in a variable called `tidy_data`.

In [14]:
data <- read_csv("data/148338_220209_095045_M057814.csv", skip=2)
education_level <- data %>% pull(response) %>% first()

data <- data %>%
    select(c(rowNo, type, stim1, stim2, stimPos, trialType, response, RT)) %>%
    filter(type != "form") %>%
    mutate(
        education_level = education_level,
        trial_number = row_number(),
        trialType = factor(trialType, levels = c("incongruent", "congruent"))
    ) %>%
    rename(trial_type = trialType)

tidy_data <- data %>%
    select(-c(rowNo, type)) %>%
    rename(stim_left = stim1, stim_right = stim2) %>%
    mutate(subject_id = 1) %>%
    mutate(correct_side = if_else(str_detect(stim_left, "small"), "left", "right")) %>%
    mutate(correct_key = if_else(correct_side == "left", "f", "j")) %>%
    mutate(correct = if_else(response == correct_key, 1, 0)) %>%
    mutate(trial_block = if_else(trial_number <= 120, 1, 2)) %>%
    mutate(trial_number = if_else(trial_number > 120, trial_number - 120, trial_number)) %>%
    rename(stim_pos = stimPos, rt = RT) %>%
    select(subject_id, trial_block, trial_number, everything())

head(tidy_data)

Rows: 241 Columns: 32
-- Column specification --------------------------------------------------------
Delimiter: ","
chr (21): type, stim1, stim2, stimPos, stimFormat, feedbackIncorrect, head, ...
dbl (11): rowNo, ITI, feedbackTime, random, ITI_ms, ITI_f, ITI_fDuration, ti...

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


subject_id,trial_block,trial_number,stim_left,stim_right,stim_pos,trial_type,response,rt,education_level,correct_side,correct_key,correct
<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<fct>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
1,1,1,croissantLarge,breadSmall,-341 0; 341 0,incongruent,j,729,College or Technical School,right,j,1
1,1,2,lampSmall,shelfLarge,-235 0; 235 0,congruent,f,625,College or Technical School,right,j,0
1,1,3,schoolbusLarge,bikeSmall,-370 0; 370 0,congruent,j,1477,College or Technical School,right,j,1
1,1,4,orangeLarge,raspberrySmall,-307 0; 307 0,congruent,j,793,College or Technical School,right,j,1
1,1,5,hammerLarge,ladderSmall,-156 0; 156 0,incongruent,j,900,College or Technical School,right,j,1
1,1,6,wheelSmall,coinLarge,-300 0; 300 0,incongruent,f,699,College or Technical School,right,j,0


### Summary statistics

In [15]:
tidy_data %>%
    summarize(rt = mean(rt), accuracy = mean(correct), error = mean(1 - correct))

rt,accuracy,error
<dbl>,<dbl>,<dbl>
622.7542,0.4916667,0.5083333


In [16]:
tidy_data %>%
    mutate(avg_error = mean(1 - correct))

subject_id,trial_block,trial_number,stim_left,stim_right,stim_pos,trial_type,response,rt,education_level,correct_side,correct_key,correct,avg_error
<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<fct>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,1,1,croissantLarge,breadSmall,-341 0; 341 0,incongruent,j,729,College or Technical School,right,j,1,0.5083333
1,1,2,lampSmall,shelfLarge,-235 0; 235 0,congruent,f,625,College or Technical School,right,j,0,0.5083333
1,1,3,schoolbusLarge,bikeSmall,-370 0; 370 0,congruent,j,1477,College or Technical School,right,j,1,0.5083333
1,1,4,orangeLarge,raspberrySmall,-307 0; 307 0,congruent,j,793,College or Technical School,right,j,1,0.5083333
1,1,5,hammerLarge,ladderSmall,-156 0; 156 0,incongruent,j,900,College or Technical School,right,j,1,0.5083333
1,1,6,wheelSmall,coinLarge,-300 0; 300 0,incongruent,f,699,College or Technical School,right,j,0,0.5083333
1,1,7,coconutSmall,blueberryLarge,-309 0; 309 0,incongruent,f,752,College or Technical School,right,j,0,0.5083333
1,1,8,pumpkinLarge,mushroomSmall,-304 0; 304 0,congruent,j,567,College or Technical School,right,j,1,0.5083333
1,1,9,croissantSmall,breadLarge,-341 0; 341 0,congruent,f,556,College or Technical School,right,j,0,0.5083333


#### Exercise 12
Using the `.by` argument in the call to `summarize`, find out if our participant showed the predicted size Stroop in reaction times and error rates.

#### Exercise 13
Using a similar logic, find out if our participant also showed a SNARC effect: Was the participant faster when the small image was on the left?

#### Exercise 14
Find out if the SNARC effect depends on trial type.

## From a single participant to a full dataset
We load all the 12 csv files in the 'data' folder, then apply the `read_csv` function to each of them using `map_dfr`. The `.id` argument creates a column that keeps the information from each file separate. This is equivalent to having a subject_id, if there is one .csv file per participant.

In [17]:
# Fetch all the files in the 'data' folder that end in .csv
raw_data <- list.files(path = 'data', pattern = ".csv$", full.names = TRUE) %>% 

  # Map the read_csv function to all of them, skipping the first 2 rows and creating a new id column called 'id' so that each file gets its own id
  # col_types = cols() just makes explicit that you want tidyverse to do its best to guess the type of each column (string, numeric, etc.)
  map_dfr(read_csv, col_types = cols(), skip = 2, .id = 'id') 

#### Exercise 15
Tidy the dataset exactly as we did for subject 1, while keeping the education-level information for each subject. Store it as `tidy_data`.

Hint: **group** the tibble before calling `first(response)`), then follow the same steps as before to obtain a tidy dataset. Use `ungroup()` to return to the tibble to the ungrouped state. In fact, in most of the exercises that follow, you will need to use grouping wisely.

```R
full_data <- raw_data %>% 
    mutate(education_level = first(response), .by = id) %>%
    ...
```

#### Exercise 16
Trials where the responses were too slow or too fast should be excluded from the analysis. (Why?)  
Exclude the trials where the response is below 200 ms or higher than 1,500 ms. How many trials were excluded?

#### Exercise 17 
Exclude participants who didn't achieve 93% overall accuracy.  
How many subjects were excluded?

#### Exercise 18
Summarize the response-time and accuracy measures by trial type to check whether there's a Stroop effect.

## Changing the format of the data: `pivot_wider`, `pivot_longer`

#### Exercise 19: `pivot_wider`
Compute the average Stroop effects for each participant. One column should be called `stroop_rt`, the other should be called `stroop_error`. Using `pull`, extract the reaction-time Stroop vector and plot its histogram.

#### Exercise 20: `pivot_longer`
Building on the output tibble in Exercise 19, remove all columns except `id`, `stroop_rt`, and `stroop_error`, then gather the two stroop columns into a single column called `measure`. This column should take one of two values for each subject (`stroop_rt` or `stroop_error`), while the `value` column should register the respective participants' stroop effect.

## Data plotting

### Rule 1: If your data is in the tidy format (one variable per column, one observation per row), plotting with `ggplot` will be very easy.

### Rule 2: No barplots.

### One possible way to do it

In [18]:
average_data <- full_data %>% summarize(rt = mean(rt), .by = c(id, trial_type)) 

ggplot(average_data, aes(x = trial_type, y = rt, fill = trial_type)) +
  geom_boxplot(width = 0.5, alpha = 0.45) +
  geom_point(size = 2) +
  geom_line(aes(group = id), color = 'gray') +         
  stat_summary(fun.data = mean_se, linewidth = 2, shape = 21, size = 1.5) +
  labs(title = "Average reaction times by trial type (ms)", x = "Trial type", y = "") +
  theme_minimal() + 
  theme(
    legend.position = "none", 
    plot.title = element_text(face = "bold", size = 20),
    axis.title = element_text(size = 18),
    axis.text = element_text(size = 16))

ERROR: Error: object 'full_data' not found
